### 基于 pandas DataFrame 的数据处理

+ 合并数据，将不同的DataFrame数据按行或列合并，形成新的DataFrame
+ 汇总数据，根据关键字对数据进行分组计算

In [2]:
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
from myfunctions import *
np.set_printoptions(precision=4)

## 数据合并

+ pandas.merge 可根据一个或多个关键字将不同DataFrame中的行和行链接，即进行横向的扩展
+ pandas.concat可以沿着一条轴将多个对象堆叠在一切


In [3]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})

+ 选项 on 给定新DataFrame的键
    + 如果不指定，系统会将多个数据中重叠的列作为键，并此为基础进行连接
    + pd.merge如果不指定合并的形式，则用inner连接，键是交集，
    + 其它方式还包括，"left","right","outer"
    + outer 得到的新表中的键是原数据键的并集

In [4]:
dfmerge1=pd.merge(df1, df2, on='key')
dfmerge2=pd.merge(df1, df2, on='key',how="left")
dfmerge3=pd.merge(df1, df2, on='key',how="outer")
side_by_side(df1,df2,dfmerge1,dfmerge2,dfmerge3)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
,key,data2
0,a,0
1,b,1


+ 按键进行连接，同一个键在表中有多行，进行两两组合

In [5]:
df1.fillna?

In [6]:
df1 = DataFrame({'key': ['b', 'b'],
                 'data1': range(2)})
df2 = DataFrame({'key': ['b',  'b' ],
                 'data2': range(2)})
merge4=pd.merge(df1, df2, how='inner')
side_by_side(df1,df2,merge4)

+ 也可以通过多个键进行合并

In [7]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
merge5=pd.merge(left, right, on=['key1', 'key2'], how='outer')
side_by_side(left,right,merge5)

+ 索引作为键合并, 使用选项 left_index=True 或者right_index=True
+ 左右表的键名称可以不同，用left_on 和 right_on 指定

In [8]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
merg1=pd.merge(left1, right1, left_on='key', right_index=True)
side_by_side(left1,right1,merg1)

### 轴向连接

+ numpy中数组连接 np.concatenate
+ pandas 中 concat

In [9]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [10]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [11]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

pd.concat连接Series也受传入的轴的影响
+ axis=0（默认）形成一个Series，axis=1则形成一个DataFrame

In [12]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [13]:
pd.concat([s1, s3, s2], axis=1,sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
f,NaN,5.0,NaN
g,NaN,6.0,NaN
c,NaN,NaN,2.0
d,NaN,NaN,3.0
e,NaN,NaN,4.0


In [14]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
pd.concat([df1, df2], axis=1, sort=False)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


+ 去掉重复数据

In [15]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
newdata1=data.drop_duplicates()
side_by_side(data,newdata1)

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4
,k1,k2
0,one,1
2,one,2


In [16]:
data['v1'] = range(7)
#不希望用所有列判断是否重复,可以指出用于判断的列
newdata2=data.drop_duplicates(['k1'])
newdata3=data.drop_duplicates(['k1', 'k2'], keep='last')
# 使用 drop_duplicates 默认会保留第一个，keep='last'选项则会保留最后一个。
side_by_side(data,newdata2,newdata3)

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6
,k1,k2,v1
0,one,1,0
3,two,3,3


## 数据分组计算

+ 对数据进行分组统计和计算
+ 使用groupby 方法

In [17]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
grouped = df['data1'].groupby(df['key1'])
print(df)
grouped.mean()

  key1 key2     data1     data2
0    a  one  0.636031 -1.934631
1    a  two  1.163516  1.955819
2    b  one  0.009166 -0.973800
3    b  two  0.263600  0.669792
4    a  one  0.698112 -0.721294


key1
a    0.832553
b    0.136383
Name: data1, dtype: float64

+ 用两列分组

In [18]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.667072
      two     1.163516
b     one     0.009166
      two     0.263600
Name: data1, dtype: float64

层次化的索引可以通过unstack转化为行列表示

In [19]:
means.unstack()

key2,one,two
key1,,
a,0.667072,1.163516
b,0.009166,0.263600


In [20]:
res1=df.groupby(['key1', 'key2']).mean()
side_by_side(df,res1)

+ 只选取部分列进行计算

In [21]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.327963
     two   1.955819
b    one  -0.973800
     two   0.669792

其它分组计算
+ 分位点
+ 使用自己的函数
+ 描述统计量 

In [22]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.070435
b    0.238157
Name: data1, dtype: float64

In [23]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.527484,3.890450
b,0.254434,1.643592


In [30]:
stocks = pd.read_csv('data/A1.csv',encoding="gbk",na_values=['--  '],skiprows=[2781])
substocks=stocks.iloc[:,:13]
substocks.head() 

,代码,名称,收入同比%%,利润同比%%,营业利润率%%,市销率,市净率,市现率,市盈(动),总金额,量比,细分行业,地区
0,1,平安银行,34.09,15.02,32.78,3.64,1.12,0.93,7.31,3.273803e+09,1.40,银行,深圳
1,2,万 科Ａ,22.72,0.77,17.35,3.13,1.80,-23.08,16.21,3.218100e+09,1.33,全国地产,深圳
2,4,国农科技,117.08,125.75,10.98,36.85,26.93,-337.10,905.17,1.750940e+08,0.79,生物制药,深圳
3,5,世纪星源,52.10,8.05,-64.57,148.17,8.32,155.10,NaN,0.000000e+00,0.00,区域地产,深圳
4,6,深振业Ａ,311.53,425.18,17.93,6.29,3.01,-20.19,24.66,5.046951e+08,0.82,区域地产,深圳


In [31]:
substocks.columns=map(lambda x: "F"+str(x),range(13))
substocks.head()

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12
0,1,平安银行,34.09,15.02,32.78,3.64,1.12,0.93,7.31,3.273803e+09,1.40,银行,深圳
1,2,万 科Ａ,22.72,0.77,17.35,3.13,1.80,-23.08,16.21,3.218100e+09,1.33,全国地产,深圳
2,4,国农科技,117.08,125.75,10.98,36.85,26.93,-337.10,905.17,1.750940e+08,0.79,生物制药,深圳
3,5,世纪星源,52.10,8.05,-64.57,148.17,8.32,155.10,NaN,0.000000e+00,0.00,区域地产,深圳
4,6,深振业Ａ,311.53,425.18,17.93,6.29,3.01,-20.19,24.66,5.046951e+08,0.82,区域地产,深圳


In [32]:
grouped=substocks.groupby("F12")

In [33]:
results=grouped.describe()
results.head()

F0                                                            \
     count           mean            std    min        25%       50%   
F12                                                                    
上海   219.0  470839.981735  216423.913350  668.0  300326.50  600610.0   
云南    30.0  251143.666667  295172.840070  538.0    1228.25    2577.5   
内蒙    25.0  396636.480000  269704.076381  426.0    2688.00  600111.0   
北京   259.0  347354.046332  245993.041606   10.0    2717.00  300379.0   
吉林    40.0  285930.725000  295532.904000   30.0     754.00  151365.0   

                             F2             ...           F9                \
           75%       max  count       mean  ...          75%           max   
F12                                         ...                              
上海   600760.50  603918.0  219.0  33.327352  ...  464146000.0  6.734183e+09   
云南   600449.75  601099.0   30.0  12.088667  ...  238602028.0  9.376749e+08   
内蒙   600295.00  601216.0   25.0   2.376400  ...  217340496.0  2.701873e+09   
北京   600580.00  603979.0  259.0  23.082008  ...  511800688.0  7.802157e+09   
吉林   600381.75  603099.0   40.0  40.845000  ...  306623152.0  1.177235e+09   

       F10                                                        
     count      mean       std  min     25%    50%     75%   max  
F12                                                               
上海   219.0  0.799772  0.452698  0.0  0.6600  0.820  1.0850  2.21  
云南    30.0  0.665000  0.394407  0.0  0.5875  0.705  0.8175  1.78  
内蒙    25.0  0.704800  0.509093  0.0  0.0200  0.780  0.9800  1.65  
北京   259.0  0.847606  0.461970  0.0  0.6800  0.860  1.0500  2.45  
吉林    40.0  0.628000  0.371243  0.0  0.5800  0.735  0.8800  1.21  

[5 rows x 80 columns]

In [34]:

results.F4.unstack()
# 该版本的pandas的quantile函数不能忽略nan,所以相应位置报错

       F12
count  上海     218.00
       云南      30.00
       内蒙      25.00
       北京     259.00
       吉林      40.00
               ...  
max    辽宁      54.08
       重庆      95.20
       陕西      82.92
       青海      32.62
       黑龙江    189.60
Length: 256, dtype: float64

In [35]:

substocks.groupby(['F11'])[['F8']].mean()

,F8
F11,
专用机械,162.937606
中成药,156.130635
乳制品,62.433750
互联网,134.174706
仓储物流,118.493333
...,...
银行,6.255625
陶瓷,133.497500
食品,62.847778


In [ ]:
+ 显然，上面的例子中数据的清理工作并不完全,需要进一步处理。
+ 关于数据处理和分组计算更详细的内容请参考 Python for Data Analysis 参考书7，9章，对照书附带的ch07.ipynb 和 ch09.ipybn进行深入学习。

## 作业

1. 将A1.csv-A3.csv 读入合并，同一个股票和合并为一行。注意，这些表格中都有相同的代码和名称列,可以用它们作为index。也可能有一些冗余的列和行，对数据进行清理。
2. 按"细分行业" 将资产分组 
   + 统计平均市盈率；
   + 用总金额作权重，计算加权市盈率；
4. 将上述计算结果合并后存为csv文件。